# Standard packages

In [1]:
import os
import pandas as pd
import time

# Specific installs

ProbKnot is included in the `RNAstructure` package.

In [2]:
method_name = "ProbKnot"

In [3]:
# download RNAstructure package
!wget -q http://rna.urmc.rochester.edu/Releases/current/RNAstructureLinuxTextInterfaces64bit.tgz
!tar xfz RNAstructureLinuxTextInterfaces64bit.tgz

# set variables for calling RNAstructure
source_path  = "/content/RNAstructure/"
data_table   = os.path.join(source_path, 'data_tables')
probknot_bin = os.path.join(source_path, 'exe', 'ProbKnot')
ct2dot_bin   = os.path.join(source_path, 'exe', 'ct2dot')

!{probknot_bin} --version

ProbKnot: Version 6.4 (December 8, 2021).
Copyright Mathews Lab, University of Rochester.


# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [4]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [5]:
# Sequences to process
yeast18lnc = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
              "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
              "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [6]:
def run_folding(fasta_name):
  out_file_name = fasta_name + '.dot'

  # make prediction
  os.system(f"export DATAPATH={data_table}; {probknot_bin} {fasta_name} tmp.cs --sequence >/dev/null 2>&1") 
  # convert ct to dot-bracket format
  os.system(f"export DATAPATH={data_table}; {ct2dot_bin} tmp.cs -1 {out_file_name} >/dev/null 2>&1") 

  return out_file_name

In [7]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

lnc_ids = yeast18lnc
print("   \t lnc \t len \t time")
for i, lnc in enumerate(lnc_ids): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(lnc_ids)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 2.0 s
2/18	snR34 	 203	 1.9 s
3/18	snR43 	 209	 1.5 s
4/18	snR44 	 211	 1.4 s
5/18	snR31 	 225	 1.7 s
6/18	snR10 	 245	 2.2 s
7/18	snR63 	 255	 2.4 s
8/18	snR11 	 258	 2.6 s
9/18	snR82 	 268	 2.8 s
10/18	snR17b 	 332	 5.3 s
11/18	snR17a 	 333	 5.3 s
12/18	snR37 	 386	 8.1 s
13/18	SCR1 	 522	 19.9 s
14/18	SRG1 	 551	 21.2 s
15/18	snR19 	 568	 27.2 s
16/18	snR30 	 606	 31.0 s
17/18	LSR1 	 1175	 220.6 s
18/18	TLC1 	 1301	 295.0 s
